In [ ]:
seed=101
records_count=10000

In [ ]:
import os
import cv2
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from keras.applications.resnet import ResNet50
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from keras.models import Sequential
from sklearn.metrics import roc_auc_score
from keras import callbacks 
from keras.callbacks import ModelCheckpoint,EarlyStopping
from  matplotlib import pyplot as plt
from keras import Model
from keras.layers import concatenate,Dense, Conv2D, MaxPooling2D, Flatten,Input,Activation,add,AveragePooling2D,GlobalAveragePooling2D,BatchNormalization,Dropout
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

#download ISIC 2019

In [ ]:
import requests
from zipfile import ZipFile

def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filename
file_name=download_file("https://isic-challenge-data.s3.amazonaws.com/2019/ISIC_2019_Training_Input.zip")
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
	# printing all the contents of the zip file
	# zip.printdir()

	# extracting all the files
	print('Extracting all the files now...')
	zip.extractall()
	print('Done!')

download meta

In [ ]:
import requests
from zipfile import ZipFile

def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filename
file_name=download_file("https://isic-challenge-data.s3.amazonaws.com/2019/ISIC_2019_Training_Metadata.csv")

In [ ]:
import requests
from zipfile import ZipFile

def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filename
file_name=download_file("https://isic-challenge-data.s3.amazonaws.com/2019/ISIC_2019_Training_GroundTruth.csv")

#dicover dataset

In [ ]:
import pandas as pd
meta=pd.read_csv("/content/ISIC_2019_Training_Metadata.csv")
meta.head()

,image,age_approx,anatom_site_general,lesion_id,sex
0,ISIC_0000000,55.0,anterior torso,NaN,female
1,ISIC_0000001,30.0,anterior torso,NaN,female
2,ISIC_0000002,60.0,upper extremity,NaN,female
3,ISIC_0000003,30.0,upper extremity,NaN,male
4,ISIC_0000004,80.0,posterior torso,NaN,male


In [ ]:
import pandas as pd
pd_meta=pd.read_csv("/content/ISIC_2019_Training_GroundTruth.csv")
pd_meta.head()
# meta
# with us =>  mel nv bcc ak bkl scc
# DF VASC UNK

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Convert Wide to Long Format
Umeta = pd.melt(pd_meta, id_vars='image', var_name='diagnosis', value_name='value').\
    sort_values('image').\
    reset_index(drop=True)

# Select rows when diagnosis == 1
Umeta = Umeta[Umeta['value'] == 1]
Umeta.drop('value', inplace=True, axis = 1)
Umeta.reset_index(inplace = True, drop = True)

In [ ]:
Umeta.head()

,image,diagnosis
0,ISIC_0000000,NV
1,ISIC_0000001,NV
2,ISIC_0000002,MEL
3,ISIC_0000003,NV
4,ISIC_0000004,MEL


In [ ]:
meta['diagnosis'] = Umeta['diagnosis']

In [ ]:
meta.head()

,image,age_approx,anatom_site_general,lesion_id,sex,diagnosis
0,ISIC_0000000,55.0,anterior torso,NaN,female,NV
1,ISIC_0000001,30.0,anterior torso,NaN,female,NV
2,ISIC_0000002,60.0,upper extremity,NaN,female,MEL
3,ISIC_0000003,30.0,upper extremity,NaN,male,NV
4,ISIC_0000004,80.0,posterior torso,NaN,male,MEL


In [ ]:
meta['diagnosis'].value_counts()

NV      12875
MEL      4522
BCC      3323
BKL      2624
AK        867
SCC       628
VASC      253
DF        239
Name: diagnosis, dtype: int64

# read meta

In [ ]:
# metadata_path="/content/ISIC_2020_Training_GroundTruth_v2.csv"
# meta = pd.read_csv(metadata_path)
meta=meta[:records_count]
images = meta['image']
target = meta['diagnosis']

In [ ]:
meta['sex']=meta['sex'].map({'female': 2, 'male': 1})
meta['sex']=meta['sex'].fillna(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
meta['age_approx'].count()

9688

In [ ]:
meta['age_approx']=meta['age_approx'].fillna(-1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
meta['age_approx'].count()

10000

In [ ]:
will_be_hot=[]
for i in meta.columns:
  if meta[i].dtypes=='O':
    print(i)
    will_be_hot.append(i)

image
anatom_site_general
lesion_id
diagnosis


#pre-processing

In [ ]:
for hot in will_be_hot:
  if hot=="image" :
    continue
  ls=meta[hot].values.tolist()
  le = preprocessing.LabelEncoder()
  meta[hot] = le.fit_transform(meta[hot]) 

In [ ]:
# #getting features from images
from keras.applications.efficientnet import EfficientNetB4
import random as rn
import tensorflow as tf
import numpy as np
from keras import backend as K

from keras import applications
from keras.applications.efficientnet import EfficientNetB4
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

# seed=1   
num_cores=1  
os.environ['PYTHONHASHSEED'] = '0'                      
np.random.seed(seed)
rn.seed(seed)
tf.random.set_seed(seed)
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=num_cores,
                              inter_op_parallelism_threads=num_cores, 
                              allow_soft_placement=True,
                              device_count = {'CPU' : 1, 'GPU' : 0})
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

#getting features from images
model = tf.keras.applications.MobileNetV2(input_shape=(380, 380, 3), include_top=False, weights='imagenet')

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
from tqdm import tqdm
for indx,i in enumerate(tqdm(meta['image'][:5000])):
    path=os.path.join("/content/ISIC_2019_Training_Input/",i)
    image=tf.keras.utils.load_img(path+'.jpg',target_size=(380,380))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = cv2.cvtColor(input_arr,cv2.COLOR_BGR2RGB)
    input_arr=np.array([input_arr])
    input_arr=input_arr/255
    features=model.predict(input_arr)
    nx, ny,_ = features[0].shape
    d2_train_dataset = features[0].reshape(nx*ny*_)
    meta['image'][indx]=d2_train_dataset

In [ ]:
from tqdm import tqdm
for indx,i in enumerate(tqdm(meta['image'][5000:])):
    path=os.path.join("/content/ISIC_2019_Training_Input/",i)
    image=tf.keras.utils.load_img(path+'.jpg',target_size=(380,380))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = cv2.cvtColor(input_arr,cv2.COLOR_BGR2RGB)
    input_arr=np.array([input_arr])
    input_arr=input_arr/255
    features=model.predict(input_arr)
    nx, ny,_ = features[0].shape
    d2_train_dataset = features[0].reshape(nx*ny*_)
    meta['image'][indx]=d2_train_dataset

In [ ]:
type(i)==int

False

In [ ]:
for indx,i in enumerate(tqdm(meta['image'])):
    if type(i)==str:
      path=os.path.join("/content/ISIC_2019_Training_Input/",i)
      image=tf.keras.utils.load_img(path+'.jpg',target_size=(380,380))
      input_arr = tf.keras.preprocessing.image.img_to_array(image)
      input_arr = cv2.cvtColor(input_arr,cv2.COLOR_BGR2RGB)
      input_arr=np.array([input_arr])
      input_arr=input_arr/255
      features=model.predict(input_arr)
      nx, ny,_ = features[0].shape
      d2_train_dataset = features[0].reshape(nx*ny*_)
      meta['image'][indx]=d2_train_dataset
      

#split data

In [ ]:
train,test0=train_test_split(meta,test_size=0.2,random_state=seed)
valid,test=train_test_split(test0,test_size=0.5,random_state=seed)

In [ ]:
train['image'].shape

(8000,)

In [ ]:
meta[:records_count]['diagnosis'].unique()

array([5, 4, 2, 3, 6, 1, 7, 0])

In [ ]:
meta['diagnosis'].unique()

array([5, 4, 2, 3, 6, 1, 7, 0])

In [ ]:
imgF=[]
for indx in train.index:
  imgF.append(train['image'][indx])
imgF_tst=[]
for indx in test.index:
  imgF_tst.append(test['image'][indx])

In [ ]:
imgF_valid=[]
for indx in valid.index:
  imgF_valid.append(valid['image'][indx])

In [ ]:
target=train['diagnosis']
x=train.drop(['diagnosis','image'],axis=1)

target_test=test['diagnosis']
x_test=test.drop(['diagnosis','image'],axis=1)

In [ ]:
target_valid=valid['diagnosis']
x_valid=valid.drop(['diagnosis','image'],axis=1)

In [ ]:
np.unique(target)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
meta_clf = RandomForestClassifier(random_state=seed)
img_clf = LogisticRegression(random_state=seed,warm_start=True)

img_clf.fit(imgF[:1000], target[:1000])
img_clf.fit(imgF[1000:2000], target[1000:2000])
img_clf.fit(imgF[2000:3000], target[2000:3000])
img_clf.fit(imgF[3000:4000], target[3000:4000])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegression(random_state=101, warm_start=True)

In [ ]:
img_clf.fit(imgF[4000:5000], target[4000:5000])
img_clf.fit(imgF[5000:6000], target[5000:6000])
img_clf.fit(imgF[6000:7000], target[6000:7000])
img_clf.fit(imgF[7000:8000], target[7000:8000])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegression(random_state=101, warm_start=True)

In [ ]:
meta_clf.fit(x, target)

RandomForestClassifier(random_state=101)

In [ ]:
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
import seaborn as sns

def multiclass_roc_auc_score(y_test, y_pred, average="weighted"):
        lb = LabelBinarizer()
        lb.fit(y_test)
        y_test = lb.transform(y_test)
        y_pred = lb.transform(y_pred)

        return  metrics.accuracy_score(y_test,y_pred)
        # return metrics.roc_auc_score(y_test, y_pred, average=average)

#meta + image in one model

In [ ]:
predicted_img_labels_tst = img_clf.predict_proba(imgF_tst)
predicted_meta_labels_tst = meta_clf.predict_proba(x_test)

In [ ]:
predicted_img_labels_valid = img_clf.predict_proba(imgF_valid)
predicted_meta_labels_valid = meta_clf.predict_proba(x_valid)

In [ ]:
# pre_img=[]
# for i in predicted_img_labels_valid:
#   pre_img.append(np.argmax(i))
# pre_meta=[]
# for i in predicted_meta_labels_valid:
#   pre_meta.append(np.argmax(i))

pre_img=[]
for i in predicted_img_labels_tst:
  pre_img.append(np.argmax(i))
pre_meta=[]
for i in predicted_meta_labels_tst:
  pre_meta.append(np.argmax(i))


In [ ]:
print('img',multiclass_roc_auc_score(pre_img,target_test)*100)
print('meta',multiclass_roc_auc_score(pre_meta,target_test)*100)

img 58.199999999999996
meta 75.8


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
print("Precision_score: ",precision_score(pre_img,target_test, average='weighted')*100)
print("Recall_score: ",recall_score(pre_img,target_test, average='weighted')*100)
print("F1_score: ",f1_score(pre_img,target_test, average='weighted')*100)

Precision_score:  65.38006056267052
Recall_score:  58.199999999999996
F1_score:  61.49497013544507


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
print("Precision_score: ",precision_score(pre_meta,target_test, average='weighted')*100)
print("Recall_score: ",recall_score(pre_meta,target_test, average='weighted')*100)
print("F1_score: ",f1_score(pre_meta,target_test, average='weighted')*100)

Precision_score:  75.99511725772594
Recall_score:  75.8
F1_score:  75.85562607746236


In [ ]:
all_x=[]
all_im=[]
all_y=[]
for i in range(len(predicted_meta_labels_valid)):
  all_x.append(predicted_meta_labels_valid[i])
  all_im.append(predicted_img_labels_valid[i])
  all_y.append(target_valid.iloc[i])

  
  all_x.append(predicted_meta_labels_tst[i])
  all_im.append(predicted_img_labels_tst[i])
  all_y.append(target_test.iloc[i])

In [ ]:
# df = pd.DataFrame(lst)
dict = {'meta': all_x, 'image': all_im, 'label': all_y} 
df = pd.DataFrame(dict)

In [ ]:
df

,meta,image,label
0,"[0.01, 0.03, 0.17, 0.0, 0.08, 0.7, 0.0, 0.01]","[6.094134044078359e-14, 1.514751498118353e-11,...",5
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[7.06305233390826e-16, 3.6796194643112944e-17,...",5
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[5.378105488970912e-12, 6.161557352240286e-12,...",5
3,"[0.0, 0.0, 0.06713854644928813, 0.0, 0.0585200...","[1.1734255957087268e-09, 1.0850505354356207e-1...",5
4,"[0.0, 0.0, 0.14882460594960598, 0.0, 0.5636865...","[1.7809379057914367e-16, 6.5759824453553015e-1...",5
...,...,...,...
1995,"[0.0, 0.0, 0.01, 0.0, 0.0, 0.99, 0.0, 0.0]","[1.7993307147368506e-07, 0.9713560335469167, 2...",5
1996,"[0.0, 0.0, 0.0, 0.0, 0.02, 0.98, 0.0, 0.0]","[2.55810835445071e-14, 1.9356590576441172e-20,...",5
1997,"[0.0, 0.0, 0.0, 0.0, 0.23, 0.77, 0.0, 0.0]","[1.9127660687912407e-23, 3.4295355638648176e-3...",5
1998,"[0.0, 0.0, 0.31, 0.52, 0.03, 0.14, 0.0, 0.0]","[3.603039018002818e-10, 1.1166382270849577e-07...",5


In [ ]:
train,test0=train_test_split(df,test_size=0.2,random_state=seed)
test,valid=train_test_split(test0,test_size=0.5,random_state=seed)

In [ ]:
x_train=train.drop(['label'],axis=1)
y_train=train['label']
x_test=test.drop(['label'],axis=1)
y_test=test['label']
x_valid=valid.drop(['label'],axis=1)
y_valid=valid['label']

In [ ]:
np.array(x_train['meta'].tolist())

array([[0.        , 0.03      , 0.18      , ..., 0.01      , 0.08      ,
        0.65      ],
       [0.        , 0.        , 0.31      , ..., 0.67      , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.80932598, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.46      , ..., 0.53      , 0.        ,
        0.        ],
       [0.        , 0.01      , 0.9       , ..., 0.01      , 0.02      ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ]])

In [ ]:
y_train

668     7
1345    5
373     5
1388    4
132     2
       ..
1599    6
1862    5
1361    5
1547    4
863     5
Name: label, Length: 1600, dtype: int64

In [ ]:
np.argmax(np.array(x_train['image'].tolist()[2]))

5

In [ ]:
pr_img=x_train['image'].tolist()
pr_meta=x_train['meta'].tolist()
both=0
only_img=0
only_meta=0
print(len(pr_img))
for indx,i in enumerate(y_train):
  # print(pr_meta[0])
  if np.argmax(pr_meta[indx])==i and np.argmax(pr_img[indx])==i:
    both+=1
  elif np.argmax( pr_meta[indx])==i:
    only_meta+=1
  elif np.argmax(pr_img[indx])==i:
    only_img+=1
print('only_img',only_img)
print('only_meta',only_meta)
print('both',both)

1600
only_img 146
only_meta 410
both 799


In [ ]:
import os 
import random as rn
import tensorflow as tf
import numpy as np
from keras import backend as K
# seed=1
num_cores=1  
os.environ['PYTHONHASHSEED'] = '0'                      
np.random.seed(seed)
rn.seed(seed)
tf.random.set_seed(seed)
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=num_cores,
                              inter_op_parallelism_threads=num_cores, 
                              allow_soft_placement=True,
                              device_count = {'CPU' : 1, 'GPU' : 0})
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

from tensorflow import keras
from keras import regularizers
model_inputs=[]
for i in range(2):
  model_inputs.append(keras.Input(shape=(8,)))

combinedInput = tf.keras.layers.concatenate(model_inputs)
# 390
x = Dense(1024,activation="relu",
          kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
          )(combinedInput)

x=Dropout(.9)(x)
predictions = Dense(8,activation="softmax",)(x)
model = Model(inputs=model_inputs,outputs=predictions)

opt= keras.optimizers.Adam(learning_rate=0.0001,epsilon=0.01)
model.compile(optimizer=opt,loss="sparse_categorical_crossentropy",metrics=['accuracy'])

# valid
model.fit([np.array(x_train['meta'].tolist()),np.array(x_train['image'].tolist())],y_train ,epochs=200,validation_data=([np.array(x_valid['meta'].tolist()),np.array(x_valid['image'].tolist())],y_valid),batch_size=16)#validation_data=(validList,validDic['diagnostic'])

print("evaluate")

Epoch 1/200
100/100 [==============================] - 2s 6ms/step - loss: 2.0520 - accuracy: 0.1931 - val_loss: 1.9667 - val_accuracy: 0.6750
Epoch 2/200
100/100 [==============================] - 0s 4ms/step - loss: 1.8904 - accuracy: 0.4888 - val_loss: 1.7999 - val_accuracy: 0.6750
Epoch 3/200
100/100 [==============================] - 0s 4ms/step - loss: 1.7118 - accuracy: 0.6575 - val_loss: 1.6327 - val_accuracy: 0.6750
Epoch 4/200
100/100 [==============================] - 0s 4ms/step - loss: 1.5485 - accuracy: 0.6837 - val_loss: 1.4725 - val_accuracy: 0.6750
Epoch 5/200
100/100 [==============================] - 0s 3ms/step - loss: 1.4011 - accuracy: 0.6906 - val_loss: 1.3352 - val_accuracy: 0.6750
Epoch 6/200
100/100 [==============================] - 0s 3ms/step - loss: 1.2836 - accuracy: 0.6906 - val_loss: 1.2280 - val_accuracy: 0.6750
Epoch 7/200
100/100 [==============================] - 0s 3ms/step - loss: 1.1931 - accuracy: 0.6900 - val_loss: 1.1510 - val_accuracy: 0.6750

In [ ]:
# test
model_loss,modelAccuracy = model.evaluate([np.array(x_test['meta'].tolist()),np.array(x_test['image'].tolist())],y_test)
print("model accuracy "+str(modelAccuracy))

7/7 [==============================] - 0s 3ms/step - loss: 0.7424 - accuracy: 0.7800
model accuracy 0.7799999713897705


In [ ]:
predictions = model.predict([np.array(x_test['meta'].tolist()),np.array(x_test['image'].tolist())], verbose=0)

In [ ]:
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report,roc_auc_score,roc_curve,auc
from tensorflow.keras.utils import to_categorical

#geting predictions on test dataset
y_pred = np.argmax(predictions, axis=1)

#getting the true labels per image 
y_true = y_test

# Creating classification report 
report = classification_report(y_true, y_pred)

print("\nClassification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.83      0.56      0.67         9
           2       0.68      0.71      0.69        24
           3       0.00      0.00      0.00         4
           4       0.50      0.52      0.51        21
           5       0.84      0.89      0.86       138
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         1

    accuracy                           0.78       200
   macro avg       0.36      0.33      0.34       200
weighted avg       0.75      0.78      0.76       200



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
print("Precision_score: ",precision_score(y_true, y_pred, average='weighted')*100)
print("Recall_score: ",recall_score(y_true, y_pred, average='weighted')*100)
print("F1_score: ",f1_score(y_true, y_pred, average='weighted')*100)

Precision_score:  74.89469387755102
Recall_score:  78.0
F1_score:  76.25651837234282


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
